In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
from time import sleep

In [2]:
params = {
    '2': {
        'p_link':'http://www.menneske.no/sudoku/2/eng/random.html?diff=1',
        'p_sol':'http://www.menneske.no/sudoku/2/eng/solution.html?number={}',
        'number':10000
    },
    '3': {
        'p_link':'http://www.menneske.no/sudoku/eng/random.html?diff=7',
        'p_sol':'http://www.menneske.no/sudoku/eng/solution.html?number={}',
        'number':10000
    },
    '4': {
        'p_link':'http://www.menneske.no/sudoku/4/eng/random.html?diff=7',
        'p_sol':'http://www.menneske.no/sudoku/4/eng/solution.html?number={}',
        'number':10000
    },
    '5': {
        'p_link':'http://www.menneske.no/sudoku/5/eng/random.html?diff=7',
        'p_sol':'http://www.menneske.no/sudoku/5/eng/solution.html?number={}',
        'number':5000
    },
    '6': {
        'p_link':'http://www.menneske.no/sudoku/6/eng/random.html?diff=5',
        'p_sol':'http://www.menneske.no/sudoku/6/eng/solution.html?number={}',
        'number':1500
    },
    '7': {
        'p_link':'http://www.menneske.no/sudoku/7/eng/',
        'p_sol':'http://www.menneske.no/sudoku/7/eng/solution.html?number={}',
        'number':18
    },
    '8': {
        'p_link':'http://www.menneske.no/sudoku/8/eng/',
        'p_sol':'http://www.menneske.no/sudoku/8/eng/solution.html?number={}',
        'number':6
    }
}

def get_sudokus(p):
    puzzle_link = p['p_link']
    solution_link = p['p_sol']
    
    req = requests.get(puzzle_link)
    c = req.content    
    soup = BeautifulSoup(c, 'html.parser')

    grid_txt = soup.find_all('div', {'class':'grid'})[0].text
    puzzle_no = grid_txt[str.find(grid_txt, 'Showing puzzle')+23:str.find(grid_txt, 'Puzzletype')]

    rows = soup.find_all('tr', {'class':'grid'})
    puzzle = []
    for row in rows:
        cols = row.find_all('td')
        for col in cols:
            txt = col.text
            if txt != '\xa0':
                puzzle.append(txt)
            else:
                puzzle.append('0')
    puzzle = ' '.join(puzzle)


    req_sol = requests.get(solution_link.format(puzzle_no))
    c = req_sol.content
    soup = BeautifulSoup(c, 'html.parser')
    rows = soup.find_all('tr', {'class':'grid'})
    solution = []
    for row in rows:
        cols = row.find_all('td')
        for col in cols:
            txt = col.text
            if txt != '\xa0':
                solution.append(txt)
            else:
                solution.append('0')
    solution = ' '.join(solution)

    return puzzle, solution

In [37]:
df = pd.DataFrame({'puzzle':[], 'solutions':[]})
df.to_csv('sudoku_3.csv', index=False, sep=',')

In [3]:
set_params = params['3']
i = 1
counter = set_params['number']
def tryAgain(retries=0):
    if retries > 20: return
    try:
        with open('sudoku_3.csv', 'a') as f:
            writer = csv.writer(f)
            global i

            while i <= counter:
                puzzle_no_list = []

                p, s = get_sudokus(set_params)

                writer.writerow([p, s])
                f.flush()
                
                i += 1
    except Exception as e:
        print(e)
        sleep(2)
        retries+=1
        tryAgain(retries)

tryAgain(0)